In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from keras.datasets import cifar10


print(f"Tensorflow {tf.__version__}")

2025-10-27 09:56:17.269226: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-27 09:56:17.272725: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-27 09:56:17.536888: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-27 09:56:20.826898: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

Tensorflow 2.20.0


In [2]:

def resnet_v1_eembc():

    inputs = tf.keras.Input(shape=(32, 32, 3))

# Block 1
    x = Conv2D(32, (3, 3), padding='same', use_bias=False)(inputs)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3), padding='same', use_bias=False)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Block 2
    x = Conv2D(64, (3, 3), padding='same', use_bias=False)(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same', use_bias=False)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Block 3
    x = Conv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)


    # Classifier
    x = Flatten()(x)
    x = Dense(256, use_bias=False)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(10, activation='softmax', use_bias=False)(x)


    # Create functional model
    model= Model(inputs, outputs)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

    # Loss function
    model.compile(
        loss='categorical_crossentropy', # loss function
        optimizer=optimizer, # learning rule
        metrics=['accuracy'] # show accuracy
    )
    print(model.summary())

    return model

In [3]:
# Download data
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()

# Normalize inputs
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

# One-hot output vectors
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, 10)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, 10)

In [4]:
model = resnet_v1_eembc()

# Train model (use one-hot labels)
history = model.fit(
    train_data, train_labels_onehot,       # training data (one-hot labels)
    batch_size=128,                        # batch size
    epochs=20,                              # Maximum number of epochs
    validation_split=0.1,                  # Percentage of training data used for validation
)

# Test model: get class predictions and evaluate with one-hot labels
predictions_keras = np.argmax(model.predict(test_data, verbose=0), axis=1)
test_loss, test_accuracy = model.evaluate(test_data, test_labels_onehot, verbose=0)
print(f"Test accuracy: {test_accuracy}")
model.save("CIFAR_v2.keras")

2025-10-27 09:56:48.738926: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 128)      │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,560 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 665,952 (2.54 MB)

 Trainable params: 665,952 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 60s 165ms/step - accuracy: 0.1080 - loss: 2.3016 - val_accuracy: 0.1004 - val_loss: 2.2941
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 55s 156ms/step - accuracy: 0.1254 - loss: 2.2754 - val_accuracy: 0.1708 - val_loss: 2.2051
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 55s 157ms/step - accuracy: 0.1944 - loss: 2.1361 - val_accuracy: 0.1914 - val_loss: 2.1196
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 55s 156ms/step - accuracy: 0.2326 - loss: 2.0575 - val_accuracy: 0.2766 - val_loss: 1.9838
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 56s 159ms/step - accuracy: 0.2536 - loss: 2.0109 - val_accuracy: 0.3046 - val_loss: 1.9530
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 57s 163ms/step - accuracy: 0.2750 - loss: 1.9687 - val_accuracy: 0.3280 - val_loss: 1.9122
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 55s 157ms/step - accuracy: 0.3011 - loss: 1.9036 - val_accuracy: 0.3428 - val_loss: 1.8211
Epoch 8/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 57s 161ms/step - accuracy: 0.3276 - lo

In [5]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [6]:
net = snn_converter.load_keras_model("CIFAR_v2.keras", show_info=True)

* Input layer: input_layer, (32, 32, 3)
* InputLayer skipped.
* Conv2D layer: conv2d, (32, 32, 32) 
* Activation skipped.
* Conv2D layer: conv2d_1, (32, 32, 32) 
* Activation skipped.
* MaxPooling2D layer: max_pooling2d, (16, 16, 32) 
* Dropout skipped.
* Conv2D layer: conv2d_2, (16, 16, 64) 
* Activation skipped.
* Conv2D layer: conv2d_3, (16, 16, 64) 
* Activation skipped.
* MaxPooling2D layer: max_pooling2d_1, (8, 8, 64) 
* Dropout skipped.
* Conv2D layer: conv2d_4, (8, 8, 128) 
* Activation skipped.
* MaxPooling2D layer: max_pooling2d_2, (4, 4, 128) 
* Dropout skipped.
* Flatten skipped.
* Dense layer: dense, 256 
    weights: (256, 2048)
    mean -0.0001473170705139637, std 0.029559915885329247
    min -0.07184025645256042, max 0.0805848017334938
* Activation skipped.
* Dropout skipped.
* Dense layer: dense_1, 10 
    weights: (10, 256)
    mean -0.0012783685233443975, std 0.09198366850614548
    min -0.21436940133571625, max 0.2178104966878891



In [7]:
predictions_snn = snn_converter.predict(test_data[:300], duration_per_sample=200)

100%|██████████| 300/300 [34:04<00:00,  6.82s/it]


In [8]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

print(classification_report(test_labels[:300], predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

              precision    recall  f1-score   support

           0       0.50      0.39      0.44        36
           1       0.71      0.50      0.59        24
           2       0.16      0.22      0.19        27
           3       0.20      0.28      0.23        29
           4       0.30      0.26      0.28        23
           5       0.20      0.39      0.27        28
           6       0.64      0.26      0.38        34
           7       0.23      0.11      0.15        27
           8       0.44      0.38      0.41        37
           9       0.49      0.60      0.54        35

    accuracy                           0.35       300
   macro avg       0.39      0.34      0.35       300
weighted avg       0.40      0.35      0.35       300

Test accuracy of the SNN: 0.3466666666666667


In [9]:
model = tf.keras.models.load_model('CIFAR_v2.keras')
loss, accuracy = model.evaluate(test_data, test_labels_onehot, verbose=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.5126 - loss: 1.3553


In [10]:
print(f"Test accuracy of the ANN: {accuracy:.4f}")
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

Test accuracy of the ANN: 0.5126
Test accuracy of the SNN: 0.3466666666666667
